In [ ]:
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import scipy.sparse as sparse
import os
import time
import datetime

In [26]:
########### FUNCIONES, DERIVADAS Y OTROS ###########
def sparse_DD_neumann(Nx, dx):
    data = np.ones((3, Nx))
    data[1] = -2 * data[1]
    diags = [-1, 0, 1]
    D2 = sparse.spdiags(data, diags, Nx, Nx) / (dx ** 2)
    D2 = sparse.lil_matrix(D2)
    # Condiciones de borde de Neumann: ajustar la primera y última fila
    D2[0, 0] = -1 / (dx ** 2)
    D2[0, 1] = 1 / (dx ** 2)
    D2[-1, -1] = -1 / (dx ** 2)
    D2[-1, -2] = 1 / (dx ** 2)
    return D2.tocsr()

def sparse_DD_periodic(Nx, dx):
    data = np.ones((3, Nx))
    data[1, :] = -2.0
    diags = [-1, 0, 1]
    D2 = sparse.spdiags(data, diags, Nx, Nx, format='lil') / dx ** 2
    D2[0, -1] = 1.0 / dx**2
    D2[-1, 0] = 1.0 / dx**2
    return D2.tocsr()

def Der(D, f):
    d_f = D @ f
    return d_f

def saving_directory(string_parameters, parameter_names):
    save_directory = disc + route + project_name
    for i in range(len(string_parameters)):
        save_directory += "/" + parameter_names[i] + "=" + string_parameters[i]
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    return save_directory

In [27]:
########### ECUACIONES DE MOVIMIENTO ###########
def equations_FD(eq, field_slices, t_i, x_grid, y_grid, parameters, operators):
    if eq == 'sine_gordon':
        U_1 = field_slices[0]
        U_2 = field_slices[1]
        DDx = operators[0]
        c = parameters[0]
        gamma = parameters[1]
        alpha = parameters[2]

        ddU = Der(DDx, np.transpose(U_1))

        F = U_2
        G = c ** 2 * ddU - gamma * U_2 + alpha * (U_1 - U_1 ** 3) / 3

        fields = np.array([F, G])

    elif eq == 'PDNLS':

        ############# Dt ψ = - (μ + iν)ψ - iα DDx ψ - iβ|ψ|^2 ψ + γψ* #############

        U_1 = field_slices[0]
        U_2 = field_slices[1]

        alpha = parameters[0]
        beta = parameters[1]
        gamma = parameters[2]
        gamma_real = gamma[0]
        gamma_imag = gamma[1]
        mu = parameters[3]
        nu = parameters[4]

        DD = operators[0]

        ddU_1 = Der(DD, U_1)
        ddU_2 = Der(DD, U_2)

        F = alpha * ddU_2 + (beta * (U_1 ** 2 + U_2 ** 2) + nu + gamma_imag) * U_2 + (gamma_real - mu) * U_1
        G = -alpha * ddU_1 - (beta * (U_1 ** 2 + U_2 ** 2) + nu - gamma_imag) * U_1 - (gamma_real + mu) * U_2

        fields = np.array([F, G])

    elif eq == 'PDNLS_complex':
        U = field_slices[0]

        alpha = parameters[0]
        beta = parameters[1]
        gamma = parameters[2]
        gamma = gamma[0]
        mu = parameters[3]
        nu = parameters[4]

        DD = operators[0]
        ddU = Der(DD, U)

        F = - (mu + 1j * nu) * U - 1j * alpha * ddU - 1j * beta * np.abs(U) * U + gamma * np.conjugate(U)

        fields = np.array([F])
    return fields

In [28]:
########### INTEGRADORES TEMPORALES ###########
def RK4_FD(eq, fields, parameters, grids, dt, Nt, operators, t_rate):
    t_grid = grids[0]  # Time grid
    x_grid = grids[1]  # Spatial grid (x)
    y_grid = grids[2]  # Spatial grid (y) if needed

    fields_history = []
    time_grid = []

    for i in range(Nt - 1):
        t = t_grid[i]
        old_fields = fields.copy()

        k_1 = equations_FD(eq, old_fields, t, x_grid, y_grid, parameters, operators)
        k_2 = equations_FD(eq, old_fields + 0.5 * dt * k_1, t + 0.5 * dt, x_grid, y_grid, parameters, operators)
        k_3 = equations_FD(eq, old_fields + 0.5 * dt * k_2, t + 0.5 * dt, x_grid, y_grid, parameters, operators)
        k_4 = equations_FD(eq, old_fields + dt * k_3, t + dt, x_grid, y_grid, parameters, operators)

        new_fields = old_fields + dt * (k_1 + 2 * k_2 + 2 * k_3 + k_4) / 6
        fields = new_fields

        # Save each t_rate amount of time
        if i % t_rate == 0:
            fields_history.append(fields.copy())
            time_grid.append(t)

    return fields, fields_history, time_grid

In [33]:
if __name__ == '__main__':

    # Definiendo ubicaciones
    project_name = '/pdnlS'
    disc = 'C:/'                                        # DISCO DE TRABAJO
    route = 'simulation_data/'                          # CARPETA DE TRABAJO
    eq = 'PDNLS'                                        # ECUACION
    t_rate = 100                                        # CADA CUANTAS ITERACIONES GUARDA
    dt = 0.01
    T = 1000
    dx = 0.25
    ies = [0.1] #np.arange(0.1, 0.25, 0.01)
    jotas = [0.15] #np.arange(0.1, 0.2, 0.01)
    [tmin, tmax, dt] = [0, T, dt]
    [xmin, xmax, dx] = [-30, 30, dx]
    t_grid = np.arange(tmin, tmax + dt, dt)
    x_grid = np.arange(xmin, xmax, dx)
    T = tmax
    Nt = t_grid.shape[0]
    Nx = x_grid.shape[0]

    ### Condiciones iniciales de carpeta particular ###
    #IC_directory = "D:/mnustes_science/simulation_data/FD/rabi_extended/alpha=13.0480/beta=1.000/mu=0.1000/nu=0.0180/sigma=12.6700/gamma=0.1570/dist=32.000"
    #U_1_init = np.loadtxt(IC_directory + "/field_real.txt", delimiter=',')[-1, :]
    #U_2_init = np.loadtxt(IC_directory + "/field_img.txt", delimiter=',')[-1, :]

    print("N° of simulations: " + str(len(ies) * len(jotas)))
    for i in ies:
        for j in jotas:
            alpha = 1.0
            beta = 1.0
            mu = 0.1
            nu = i
            gamma_0 = j

            ### Condiciones iniciales random para patrones (OJO: Dos patrones estables por cada set de parametros) ###
            U_1_init = 0.01 * np.random.rand(Nx)
            U_2_init = 0.01 * np.random.rand(Nx)

            alpha_str = f"{alpha:.{4}f}"
            beta_str = f"{beta:.{4}f}"
            gamma_str = f"{gamma_0:.{4}f}"
            mu_str = f"{mu:.{4}f}"
            nu_str = f"{nu:.{4}f}"

            if i == ies[0] and j == jotas[0]:
                print('gamma = ' + gamma_str)
                print('mu = ' + mu_str)
                print('nu = ' + nu_str)
            print('### i = ' + str(i) + ' ###')
            print('### j = ' + str(j) + ' ###')

            ### Empaquetamiento de parametros, campos y derivadas para integración ###
            L = xmax - xmin
            D2 = sparse_DD_neumann(Nx, dx)
            operators = np.array([D2])
            fields_init = [U_1_init, U_2_init]
            grids = [t_grid, x_grid, 0]

            gamma_complex = gamma_0 #* np.exp(- x_grid ** 2 / (2 * sigma ** 2))
            gamma_real = np.real(gamma_complex)
            gamma_img = np.imag(gamma_complex)
            gamma = [gamma_real, gamma_img]
            parameters = [alpha, beta, gamma, mu, nu]   ## Si se usa PDNLS_complex en vez de gama, acá va gamma_complex

            ### INTEGRACIÓN TEMPORAL ###
            now = datetime.datetime.now()
            print('Hora de Inicio: ' + str(now.hour) + ':' + str(now.minute) + ':' + str(now.second))
            time_init = time.time()

            final_fields, fields_history, time_grid = RK4_FD(eq, fields_init, parameters, grids, dt, Nt, operators, t_rate)

            now = datetime.datetime.now()
            print('Hora de Término: ' + str(now.hour) + ':' + str(now.minute) + ':' + str(now.second))
            time_fin = time.time()
            print('TIEMPO TOTAL: ' + f"{(time_fin - time_init):.{3}f}" + ' seg')
            print('##############################')

            #REOBTENIENDO CAMPOS FINALES
            U1_light = np.array(fields_history)[:, 0]
            U2_light = np.array(fields_history)[:, 1]
            U_complex = U1_light + 1j * U2_light
            t_light = time_grid

            ## Paso final como condición inicial de siguiente simulación ##
            #U_1_init = U1_light[-1, :]
            #U_2_init = U2_light[-1, :]

            # VARIABLES FINALES A GUARDAR
            modulo_light_1 = np.absolute(U_complex)
            arg_light_1 = np.angle(U_complex)
            arg_light_1 = (2 * np.pi + arg_light_1) * (arg_light_1 < 0) + arg_light_1 * (arg_light_1 > 0)

            # GUARDANDO DATOS
            params_names = ["alpha", "beta", "gamma", "mu", "nu"]
            str_params = [alpha_str, beta_str, gamma_str, mu_str, nu_str]
            parameters_np = np.array([alpha, beta, gamma_0, mu, nu])
            save_directory = saving_directory(str_params, params_names)

            np.savetxt(save_directory + '/field_real.txt', U1_light, delimiter=',')
            np.savetxt(save_directory + '/field_img.txt', U2_light, delimiter=',')
            np.savetxt(save_directory + '/parameters.txt', parameters_np, delimiter=',')
            np.savetxt(save_directory + '/parameter_names.txt', np.array(params_names), fmt="%s")
            np.savetxt(save_directory + '/X.txt', x_grid, delimiter=',')
            np.savetxt(save_directory + '/T.txt', t_light, delimiter=',')

            # GUARGANDO GRAFICOS

            pcm = plt.pcolormesh(x_grid, t_light, modulo_light_1, cmap='viridis', shading='auto')
            cbar = plt.colorbar(pcm, shrink=1)
            cbar.set_label('$|A|$', rotation=0, size=20, labelpad=-27, y=1.1)
            plt.xlim([x_grid[0], x_grid[-1]])
            plt.xlabel('$x$', size='20')
            plt.ylabel('$t$', size='20')
            plt.grid(linestyle='--', alpha=0.5)
            plt.savefig(save_directory + '/module_spacetime.png', dpi=300)
            plt.close()

            pcm = plt.pcolormesh(x_grid, t_light, arg_light_1, cmap='viridis', shading='auto')
            cbar = plt.colorbar(pcm, shrink=1)
            cbar.set_label('$arg(A)$', rotation=0, size=20, labelpad=-20, y=1.1)
            plt.xlim([x_grid[0], x_grid[-1]])
            plt.xlabel('$x$', size='20')
            plt.ylabel('$t$', size='20')
            plt.grid(linestyle='--', alpha=0.5)
            plt.savefig(save_directory + '/arg_spacetime.png', dpi=300)
            plt.close()

            pcm = plt.pcolormesh(x_grid, t_light, U1_light, cmap='viridis', vmin=-np.amax(U1_light), vmax=np.amax(U1_light), shading='auto')
            cbar = plt.colorbar(pcm, shrink=1)
            cbar.set_label('$A_R(x, t)$', rotation=0, size=20, labelpad=-27, y=1.1)
            plt.xlim([x_grid[0], x_grid[-1]])
            plt.xlabel('$x$', size='20')
            plt.ylabel('$t$', size='20')
            plt.grid(linestyle='--', alpha=0.5)
            plt.savefig(save_directory + '/real_spacetime.png', dpi=300)
            plt.close()

            pcm = plt.pcolormesh(x_grid, t_light, U2_light, cmap='viridis', vmin=-np.amax(U1_light), vmax=np.amax(U1_light), shading='auto')
            cbar = plt.colorbar(pcm, shrink=1)
            cbar.set_label('$A_I(x, t)$', rotation=0, size=20, labelpad=-27, y=1.1)
            plt.xlim([x_grid[0], x_grid[-1]])
            plt.xlabel('$x$', size='20')
            plt.ylabel('$t$', size='20')
            plt.grid(linestyle='--', alpha=0.5)
            plt.savefig(save_directory + '/img_spacetime.png', dpi=300)
            plt.close()
    print('##########  FIN  ##########')

N° of simulations: 1
gamma = 0.1500
mu = 0.1000
nu = 0.1000
### i = 0.1 ###
### j = 0.15 ###
Hora de Inicio: 0:1:31
Hora de Término: 0:1:43
TIEMPO TOTAL: 12.566 seg
##############################
##########  FIN  ##########
